In [10]:
#Need to be published as a py file
# import firebase_admin
# from firebase_admin import credentials
# from firebase_admin import firestore

# cred = credentials.Certificate("./reinlife-915bd-firebase-adminsdk-hydd2-96cf9b9942.json")
# if not firebase_admin._apps:
#     firebase_admin.initialize_app(cred)
# db = firestore.client()
import ReinLifeResearcher
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, firestore
cred = credentials.Certificate("./flutternotification-ebd50-firebase-adminsdk-dum2k-315e67534e.json")
#firebase_admin.initialize_app(cred)
db = firestore.client()

In [33]:


def researcher_RL_code(Userid):
    """
    researcher defined ML code
    This should be considered as a template
    This code takes in user information(answers) of one user, and output a action (for now, a message)
    """
    
    #get_answers should be considered as an API which allow researchers to customize the information they want
    #In this example, we are getting all answers for that user for simplicity
    answers=ReinLifeResearcher.get_answers(Userid,'all')
    
    #Here, we should better check answers to make sure the format is correct!
    
    def answers_to_state_reward(answers):
        """
        Turn user answers into state and reward
        In this example, we are calculating the proportion of 'broccoli' and 'asparagus'
        """
        from collections import Counter
        #print(answers)
        food_names = [item[0] for item in answers]
        #print(food_names)
        counts = Counter(food_names)
        
        return counts
    def RL_algorithm(state_reward):
        """
        A dummy RL method. To be implemented by researchers
        The example is simply counting the number of having greens as meal
        """
        ngreens=state_reward['broccoli']+state_reward['asparagus']
        if ngreens>=3:
            action=f"Ate greens for {ngreens} times, good job eating your greens"
        else:
            action=f'Ate greens for {ngreens} times, try to eat more greens'
        return action
    
    state_reward=answers_to_state_reward(answers) 
    #print(state_reward)
    action=RL_algorithm(state_reward)
    #print(action)
    
    return action  
def push_notification(Userid, message):
    """
    This is a function that need to be implemented
    This function pushes a notification with a specific message to the front end
    """
    print("notification to be implemented: "+message)
    notification2db(Userid, message)
    
#We still need a scheduler to set when to push notification, when to run RL algorithm


#A function that set questionnaire to answer
#A function that push notification as a reminder to answer the questionnaire
#A function that do loop for all the users in an experiment
def RL_all_useres():
    Userlist=ReinLifeResearcher.list_all_userid()
    for Userid in Userlist:
        print(f"Now, doing RL for User:{Userid}")
        actionmessage=researcher_RL_code(Userid)
        push_notification(Userid,actionmessage)
        


In [34]:
def notification2db(Userid, message):
    """
    function for saving notification to database
    """
    collection_notification=db.collection('Users').document(Userid).collection('notification record')
    collection_notification.add({
        'message': message,
        'time': datetime.now() 
    })

In [35]:
notification2db('caZSPlKe5eUPEkxqwkjp', "this is a test message: eat greens!")

In [37]:
ReinLifeResearcher.set_experiment()
ReinLifeResearcher.set_questionnaires()
ReinLifeResearcher.list_all_userid()
ReinLifeResearcher.get_answers('caZSPlKe5eUPEkxqwkjp','all')

['IL8CRlPNHy5sGcWZkwBY', 'caZSPlKe5eUPEkxqwkjp']
get User:caZSPlKe5eUPEkxqwkjp answers: all
['asparagus', 5, 'No']
['burger', 5, 'No']


[['asparagus', 5, 'No'], ['burger', 5, 'No']]

In [38]:
userids=ReinLifeResearcher.list_all_userid()
userids

['IL8CRlPNHy5sGcWZkwBY', 'caZSPlKe5eUPEkxqwkjp']


['IL8CRlPNHy5sGcWZkwBY', 'caZSPlKe5eUPEkxqwkjp']

In [39]:
answers=ReinLifeResearcher.get_answers('caZSPlKe5eUPEkxqwkjp','all')
answers

get User:caZSPlKe5eUPEkxqwkjp answers: all
['asparagus', 5, 'No']
['burger', 5, 'No']


[['asparagus', 5, 'No'], ['burger', 5, 'No']]

In [40]:
answers=ReinLifeResearcher.get_answers('IL8CRlPNHy5sGcWZkwBY','all')
answers

get User:IL8CRlPNHy5sGcWZkwBY answers: all


[]

In [41]:
food_names=researcher_RL_code('caZSPlKe5eUPEkxqwkjp')

get User:caZSPlKe5eUPEkxqwkjp answers: all
['asparagus', 5, 'No']
['burger', 5, 'No']


In [25]:
food_names

'Ate greens for 0 times, try to eat more greens'

In [44]:
RL_all_useres()

['IL8CRlPNHy5sGcWZkwBY', 'caZSPlKe5eUPEkxqwkjp']
Now, doing RL for User:IL8CRlPNHy5sGcWZkwBY
get User:IL8CRlPNHy5sGcWZkwBY answers: all
notification to be implemented: Ate greens for 0 times, try to eat more greens
Now, doing RL for User:caZSPlKe5eUPEkxqwkjp
get User:caZSPlKe5eUPEkxqwkjp answers: all
['asparagus', 5, 'No']
['burger', 5, 'No']
notification to be implemented: Ate greens for 1 times, try to eat more greens


In [43]:
import schedule
import time

def my_task():
    print("Task executed!")
schedule.clear()
# Schedule the task
schedule.every(60).seconds.do(lambda: RL_all_useres('Foodeat12345'))

# Run the pending tasks in a loop for a fixed number of times
while True:
    schedule.run_pending()


KeyboardInterrupt: 

In [ ]:
import threading
def run_schedule():
    while True:
        schedule.run_pending()
        time.sleep(1)
thread = threading.Thread(target=run_schedule)
thread.start()
while True:
    time.sleep(1)

In [18]:
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.jobstores.base import ConflictingIdError
import time
scheduler = BackgroundScheduler()
scheduler.start()
def task1():
    print("Task 1 executed!")
    if some_condition():
        add_new_task()
scheduler.add_job(task1, 'interval', seconds=10, id='task1_job')


   
while True:
    time.sleep(1)

Job "task1 (trigger: interval[0:00:10], next run at: 2023-10-22 13:28:54 EDT)" raised an exception
Traceback (most recent call last):
  File "/Users/boerzhang/opt/anaconda3/envs/reinlife/lib/python3.9/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/95/kqnc1xrn05zcmk1f2dtc1ldc0000gp/T/ipykernel_55175/1745338130.py", line 9, in task1
    if some_condition():
NameError: name 'some_condition' is not defined


Task 1 executed!


Job "task1 (trigger: interval[0:00:10], next run at: 2023-10-22 13:28:55 EDT)" raised an exception
Traceback (most recent call last):
  File "/Users/boerzhang/opt/anaconda3/envs/reinlife/lib/python3.9/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/95/kqnc1xrn05zcmk1f2dtc1ldc0000gp/T/ipykernel_55175/3787025921.py", line 9, in task1
    if some_condition():
NameError: name 'some_condition' is not defined


Task 1 executed!


Job "task1 (trigger: interval[0:00:10], next run at: 2023-10-22 13:29:04 EDT)" raised an exception
Traceback (most recent call last):
  File "/Users/boerzhang/opt/anaconda3/envs/reinlife/lib/python3.9/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/95/kqnc1xrn05zcmk1f2dtc1ldc0000gp/T/ipykernel_55175/1745338130.py", line 9, in task1
    if some_condition():
NameError: name 'some_condition' is not defined


Task 1 executed!


Job "task1 (trigger: interval[0:00:10], next run at: 2023-10-22 13:29:05 EDT)" raised an exception
Traceback (most recent call last):
  File "/Users/boerzhang/opt/anaconda3/envs/reinlife/lib/python3.9/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/95/kqnc1xrn05zcmk1f2dtc1ldc0000gp/T/ipykernel_55175/3787025921.py", line 9, in task1
    if some_condition():
NameError: name 'some_condition' is not defined


Task 1 executed!


KeyboardInterrupt: 

In [19]:
while True:
    time.sleep(1)

Job "task1 (trigger: interval[0:00:10], next run at: 2023-10-22 13:29:34 EDT)" raised an exception
Traceback (most recent call last):
  File "/Users/boerzhang/opt/anaconda3/envs/reinlife/lib/python3.9/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/95/kqnc1xrn05zcmk1f2dtc1ldc0000gp/T/ipykernel_55175/1745338130.py", line 9, in task1
    if some_condition():
NameError: name 'some_condition' is not defined


Task 1 executed!


Job "task1 (trigger: interval[0:00:10], next run at: 2023-10-22 13:29:35 EDT)" raised an exception
Traceback (most recent call last):
  File "/Users/boerzhang/opt/anaconda3/envs/reinlife/lib/python3.9/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/95/kqnc1xrn05zcmk1f2dtc1ldc0000gp/T/ipykernel_55175/3787025921.py", line 9, in task1
    if some_condition():
NameError: name 'some_condition' is not defined


Task 1 executed!


KeyboardInterrupt: 

In [1]:
import subprocess

subprocess.Popen(['ls'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

<Popen: returncode: None args: ['ls']>